# 번역기를 만들어보자

## 기계번역 이야기

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-15-1.machine_translation.max-800x600.png)

1940년대부터 1980년대까지의 기계번역은 **규칙 기반의 기계 번역(Rule-based Machine Translation)**이 주를 이루었습니다. 예를 들어, 2차 세계 대전 시절 미국이 러시아어를 영어로 번역하고자 했을 때, 러시아 문장에서 단어 하나 하나의 의미를 분석하고 그 단어들이 어떻게 문법적으로 연결되었는지를 분석하고 그 다음 구조 상의 의미를 분석한 뒤에 그제서야 영어와 의미적으로 맞게 단어들을 분리하고 마지막으로 영어 문법에 맞게 순서를 맞추고.. 꽤 복잡한 과정을 거쳤었죠.

![Img](https://aiffelstaticprd.blob.core.windows.net/media/original_images/E-15-2.ibm_paper.png)

1980년대 후반에 들어서 IBM에서 규칙 기반에서 벗어나려는 시도가 있었습니다. 언어학 기반의 규칙이 아닌 통계로 구현하려는 시도였는데요, 이를 **통계적 기걔 번역(Statistical Machine Translation)**이라고 합니다. 1980년대 당시 규칙 기반의 접근이 얼마나 답답했는지 그 팀의 리더 프레데리 제리닉은 이 당시 "매번 내가 언어학자를 해고할 때 마다, 언어 인식기의 성능이 올라갔다"라는 말을 남겼다는 소문이 있죠. 이 방법은 2010년 중반까지 주요한 접근으로 사용되었습니다.

통계기반 번역에 여러 머신러닝이 사용되었지만 딥러닝은 아니었습니다. 몇 차례 인공 신경망으로 번역을 제안하는 논문은 있었지만, 훈련 데이터도 작고 신경망의 크기도 작아서 주목받지 못했습니다. 그러나 2010년 중반에 들어 알고리즘과 하드웨어의 발전으로 딥러닝이 빛을 발하면서 달라졌습니다.

구글은 2016년 9월, 자신들의 구글 번역기에 **신경망 기계 번역(Neural Machine Translation, NMT)**을 도입하면서 획기적인 성능 개선을 이루었다고 발표했습니다. 통계기반 번역에서 신경망 기계번역으로 변경되면서 한층 더 높은 수준의 번역 능력을 가질 수 있게 되었는데요, 이때 사용된 신경망이 **seq2seq**입니다. 이번 시간에는 신경망 seq2seq를 이용하여 기계번역기를 만들어봅시다.

## 시퀀스를 처리하는 RNN

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-15-3.rnn_effectiveness.max-800x600.jpg)

앞서 시쿠너스 데이터를 처리하는 RNN에 대해서 배웠습니다. RNN은 어떻게 활용하느냐에 따라서 다양한 어플리케이션에서 사용할 수 있습니다.

위 그림은 다양한 RNN의 방법을 보여주고 있습니다. 기계 번역기 구현에서는 기본적으로 4번을 활용해야 합니다. 그 이유는 *'사람 말은 끝까지 들어봐야 한다'*는 말과 연관이 있는데요, 번역 또는 통역이라고 하는 것은 전체 문장을 모두 읽거나 듣고나서야 할 수 있기 때문입니다. 번역이나 통역에 5번을 사용하면 아직 사람 말도 다 하지 않았는데 단어 하나를 들을 때마다 그 순간을 번역해야 하는데, 그럼 문맥에 맞는 말이 나올 가능성이 적어지겠죠?

## seq2seq 

### seq2seq의 인코더-디코더 구조

앞서 번역기는 다양한 RNN의 유형 그림중에서도 4번을 사용한다고 했습니다. 사실 여기에서 조금 더 깊게 살펴보자면 약간 다릅니다. 아래 그림은 번역기의 기본 구조인 seq2seq 입니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-15-4.seq2seq.max-800x600.jpg)

seq2seq는 두개의 RNN 아키텍처를 연결한 구조입니다. 입력 문장을 받는 RNN을 **인코더**라고 하고, 두번째 RNN을 **디코더**라고 합니다.

아래 그림에서 Encoder는 Feature Extractor의 역할을 합니다. 어떤 데이터 $X$를 해석하기 위한 저차원의 feature vector $z$를 만들어냅니다. 반면에 Decoder는 저차원의 Feature $z$로부터 정보를 복원해서 다시 어떤 데이터 $X'$를 재생성하는 역할을 합니다.

![IMG](https://aiffelstaticprd.blob.core.windows.net/media/images/E-15-5.encdec.max-800x600.png)

우리가 오늘 살펴볼 seq2seq 모델은 위 인코더-디코더 모델에서 인코더와 디코더 모델이 모두 RNN인 경우라고 볼 수 있을 것입니다. 그렇다면 seq2seq의 feature vector는 무엇일까요? 바로 인코더 RNN이 입력 문장을 해석해서 만들어 낸 hidden state 벡터일 것입니다. 즉, A언어의 문장 X를 z라는 hidden state로 해석한 후 z를 다시 B 언어의  문장 Y로 재생성 하는 것입니다. 그러므로 인코더에서 필요한 것은 *마지막 time step의 hidden state*입니다. 그리고 이를 두번째 RNN인 디코더에 전달합니다.

디코더는 인코더의 마지막 time step의 hidden state를 전달받아 자신의 초기 Hidden state로 하고, 출력 문장을 생성해내기 시작합니다. 여기서는 특수 문자를 사용해서 출력 문장의 시작과 종료를 알려주어야 합니다. 위의 그림에서는 `_GO`와 `EOS`가 각각 시작 문자와 종료 문자에 해당합니다. 문헌에 따라서는 `SOS`와 `EOS`라고도 하는데, `SOS`는 start of sequence를 의미하며, `EOS`는 end of sequence를 의미합니다.

### Conditional Language Model

문장 생성기(Text Generator) 모델을 만들어 보았다면, 그러한 문장 생성기는 **언어 모델(Language Model)**을 구현한 것이라는 것을 알고 있을 것입니다.

언어 모델이란 n-1개의 단어 시퀀스 $w_1, \cdots, w_{n-1}$가 주어졌을 때, N번째 단어 $w_n$으로 무엇이 올지를 예측하는 확률 모델입니다. 파라미터 $\theta$로 모델링하는 언어 모델을 다음과 같이 표현할 수 있습니다.

$$P(w_n | w_1, …, w_{n-1};\theta )$$

우리가 알고 있는 RNN 계열의 모델들은 이러한 언어 모델을 구현하기에 적함한 것들입니다.

그런데, 언어 모델에 기반한 문장 생성기가 가지고 있는 한 가지 문제점이 있습니다. 그것은 바로, *어떤 말을 만들고 싶을지를 제어할 수 없다*는 점입니다. RNN 모델이 확률적으로 그 다음 나올 단어들을 순차적으로 만들어 나가게 되는데, 그것을 상황에 맞게 제어할 수 있다면 아주 유용할 것입니다. 그래서 사람들은 위 언어 모델을 **조건적 언어 모델(Conditional Language Model)**의 개념을 생각하게 됩니다. 말하자면 아래와 같은 형태가 될 것입니다.

$$P(w_n | w_1, …, w_{n-1}, c;\theta )$$

이 식과 다르게 $c$라는 것이 하나 더 붙었지요? 이 $c$를 이용해 기계에게 '아무 문장이나 만들지 말고 c에 적합한 문장을 만들어'라고 주문하고 싶은 것입니다.

기계번역이야말로 가장 대표적인 Conditional Language Model의 사례가 될 것입니다. 'X라는 영어 문장을 Y라는 프랑스어 문장으로 번역해!'라는 것은 바꾸어 말하면, '프랑스어 문장 Y를 만들어봐, 단 그 문장은 영어로는 X라는 뜻이어야 해'라는 뜻이 됩니다. 그런데 이 조건을 어떻게 문장 생성기에 넣어줄까요? 그렇습니다. 이 문장 X를 해석해서 $c$로 만드는 인코더를 또 다른 RNN으로 만드는 것입니다. 그렇게 만든 $c$를 다시 문장생성기인 디코더 RNN에 입력으로 넣어주는 모델을 만들어 낸 것이 바로 오늘 다루게 될 seq2seq입니다.



## 교사 강요 (teacher forcing)

seq2seq는 훈련 과정과 테스트 과정에서의 동장 방식이 다르다는 특징이 있습니다. 이전 스텝의 그림을 보면 디코더 RNN은 이전 time step의 출력을 현재 time step의 입력으로 사용한다는 특징을 가지고 있습니다. 그런데 이는 테스트 과정에서의 이야기이고, 훈련 과정은 조금 다른 방식을 사용합니다. 그 이유는 훈련 과정에서 이전 time step이 잘못된 예측을 한다면 이를 입력으로 한 현재 time step의 예측도 잘못될 수 있기 때문입니다. 

![img](https://aiffelstaticprd.blob.core.windows.net/media/original_images/E-15-6.teacher_forcing.png)


훈련 과정에서는 실제 정답 시퀀스를 알고 있는 상황이므로 이전 Time step의 예측값을 현재 Time step의 입력으로 사용하는 것이 아니라 이전 time step의 실제 값으로 사용할 수 있습니다. 이 작업을 **교사 강요(teacher forcing)**이라고 합니다. 이 기법은 seq2seq 뿐만 아니라 sequence 데이터의 생성모델에서 일반적으로 사용되는 기법이기도 합니다. 물론, 이는 모델이 훈련 데이터 외의 결과를 생성해내는 능력을 기르는 데에 조금 방해가 될 수 있다는 단점도 존재합니다.

## 단어 수준 VS 문자 수준

seq2seq **단어 수준(word level)** 또는 **문자 수준(character level)** 두 가지 방법으로 구현할 수 있습니다. 단어 수준이라고 함은 각 RNN의 time step의 입출력 단위가 단어 수준이라는 의미이고, 문자 수준이라 함은 RNN의 time step의 입출력 단위가 문자 수준, 영어에서는 알파벳 단위입니다.

실제 구현 자체는 문자 수준이 좀 더 쉬운데, 그 이유는 단어 수준 seq2seq의 경우 매 time step에서 디코더 RNN이 출력을 결정할 때, 훈련 데이터에 있는 전체 단어 집합의 수가 30만개라면 30만개 중 하나를 선택해야 하므로 출력층의 크기가 무려 30만이어야 합니다.

하지만 문자 수준으로 구현하면 영문자 알파벳은 26개에 불과하므로 출력층의 크기가 굉장히 작아집니다. 여기에 대, 소문자를 구분하지 않고 특수문자를 포함한다고 하더라도 출력층의 크기는 100을 넘지 않습니다.

그렇다면 단어 수준의 번역과 문자 수준의 번역 둘 중 어느 쪽이 더 유리할까요?

두 방법에는 장단점이 있습니다. 그리고 그 장단점은 서로 trade-off 관계이기도 합니다.

단어 수준의 번역을 위해서 사전을 구축한다고 생각해봅시다. 가장 극단적인 경우에 해당하는 한국어의 예를 들어봅시다.

> 먹다, 먹는다, 먹고, 먹을까,,

'먹다'라는 단어 하나에 이렇게 많은 변종이 있고, 이 의미가 전부 아주 약간 다릅니다. 이렇게 따지면 엄청나게 큰 단어 사전이 필요하게 됩니다. 영어에도 이런 문제가 있겠지만 한국어에 비하면 그 변화가 그리 심하지는 않습니다. 또 하나의 단어 수준의 접근의 어려움은 띄어쓰기의 문제입니다. 특히 띄어쓰기가 많이 생략되는 한국어, 일본어 같은 언어들에 있어 이러한 전처리가 큰 어려움의 원인이 됩니다.

그렇다면 문자 수준으로 번역하면 이런 문제가 없이 아주 깔끔하게 해결되겠죠? 그러나 단어를 문자 수준으로 쪼갠다는 것은 단어 안에 내재된 정보가 소실된다는 것을 의미합니다. 즉, 기계가 데이터를 통해 글자가 단어를 이루는 패턴까지 학습해야 한다는 것입니다. 그래서 충분한 데이터가 확보되지 않았다면 일반적으로 문자 수준의 번역이 단어 수준의 번역보다 품질이 떨어집니다.

하지만, 최신 자연어처리 흐름은 단어 수준이나 문자 수준의 번역이 아닌, 그 사이의 *subword 기반의 번역*이 주를 이루고 있습니다. 이에 대해서는 나중에 다루어봅시다.

이번 시간에는 전처리와 훈련 시간을 고려하여 문자 수준으로 구축해보겠습니다. 그리고 일단 문자 수준 번역기를 구축하고 나면 단어 수준 모델로 변환하는 것은 굉장히 쉽습니다. 이는 나중에 따로 연습해보세요!

---

## 데이터 전처리

이번 시간에 사용할 데이터는 <https://www.manythings.org/anki/> 에서 다운로드 하도록 합시다. 여기서는 프랑스어와 영어의 병렬 코퍼스인 `fra-eng.zip`을 다운받아 사용합니다. 아래 커맨드로 다운로드 받을 수 있습니다.